In [3]:
import json
from CAT.mips.ball_tree import BallTree,search_metric_tree_c
import numpy as np
import time
from tqdm import tqdm
import heapq
import os
dataset='assistment'
# dataset='ifytek'
stg="MFI"
cdm='irt'
postfix='30'
secat=True
if secat:
    dissimilarity_partition=True
    last_leaf=True
else:
    dissimilarity_partition=False
    last_leaf = False
path_prefix = os.path.abspath('../../')
ball_trait = json.load(open(f"{path_prefix}/data/{dataset}/{stg}/ball_trait{postfix}.json", 'r'))
item_label = json.load(open(f"{path_prefix}/data/{dataset}/MFI/item_label.json", 'r'))
query = json.load(open(f'{path_prefix}/data/{dataset}/{stg}/query.json', 'r'))

if dataset=='ifytek':
    model = BallTree(dict(zip(range(len(ball_trait)),list(zip(item_label,ball_trait)))),dissimilarity_partition=dissimilarity_partition,threshold=15*20)
else:
    model = BallTree(dict(zip(range(len(ball_trait)),list(zip(item_label,ball_trait)))),dissimilarity_partition=dissimilarity_partition,threshold=20)



In [4]:
starttime= time.time()
efficient = True
times=[]
counts=[]
for qs in query:
    tested_set=set()
    leaves_candidates=set()
    for k,q in enumerate(qs):    
        s_t=time.time()
        if efficient:
            res_q={'qid':-1,'quantity':-1,'leaves':{}}
            if last_leaf:
                last=False
                if k>1:
                    for lq in qs[:k-1]:
                        tmp_cos = (np.array(q)).dot(np.array(lq)) / (np.linalg.norm(lq) * np.linalg.norm(q))
                        if tmp_cos>0.95:
                            last=True
                # leaves_candidates_inl = list(leaves_candidates-tested_set)
                if last:
                    tmp_ip = [np.dot(np.array(q), np.array(ball_trait[qid])) for qid in leaves_candidates]
                    qip = heapq.nlargest(1, tmp_ip)
                    count=len(leaves_candidates)
                    # qid = leaves_candidates_inl[list(map(tmp_ip.index, qip))[0]]
                else:
                    count = search_metric_tree_c(res_q, tested_set,np.array(q),model)
                    leaves_candidates.update(set(res_q['leaves'].keys()))
                tested_set.add(res_q['qid'])
            else:
                count = search_metric_tree_c(res_q, tested_set,np.array(q),model)  
                tested_set.add(res_q['qid'])
            counts.append(count)
        else:
            tmp = [np.dot(q,i_emb) for i_emb in np.array(ball_trait)]
            # print(max(tmp))
            qips = heapq.nlargest(k+1, tmp)
            qids = list(map(tmp.index, qips))
        times.append(time.time()-s_t)
endtime= time.time()
sec = (endtime - starttime)
print(sec/len(times)*10)
# print(times)
print(max(times))
print(counts)

0.004262716212171189
0.001220703125
[33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33, 33, 13, 13, 13, 13, 13, 13, 13, 13, 33,

In [5]:
len(counts)

470

In [6]:
sum(counts)/len(counts)

17.340425531914892

In [7]:
import numpy as np
import pandas as pd
# len(times)
s_time=[]
tmp=[]
for i,t in enumerate(times):
    tmp.append(t)
    if len(tmp)==10:
        s_time.append(tmp)
        tmp=[]
s_time=np.array(s_time)
s_time = np.average(s_time, axis=0, weights=None, returned=False)
s_time
# s_time/29*47
res=[]
a_time=0
for t in s_time:
    # print(t)
    # break
    a_time+=t
    res.append(a_time)
# print(s_time.shape)
df = pd.DataFrame(res)
df = df.transpose()

if secat:
    df.to_csv(f'data/secat_{dataset}_{stg}.csv')
else:
    df.to_csv(f'data/balltree_{dataset}_{stg}.csv')

In [8]:
df

,0,1,2,3,4,5,6,7,8,9
0,0.00085,0.00165,0.001862,0.002099,0.002366,0.002671,0.003009,0.00338,0.003785,0.004226
